In [1]:
import xml.etree.ElementTree as ET # Импортируем модуль ElementTree

Для работы со структурой файла menu.xml считаем его содержимое в переменную tree, выполнив код:

In [3]:
import xml.etree.ElementTree as ET # Импортируем модуль ElementTree
tree = ET.parse('data/menu.xml')

КОРЕНЬ

Запишем в переменную root корневой узел дерева tree и посмотрим, как выглядит содержимое переменной root, для чего выполним код:

In [5]:
root = tree.getroot()
display(root)
display(type(root))

<Element 'menu' at 0x000002C4034C0EA0>

xml.etree.ElementTree.Element

ПОТОМКИ

Для того чтобы посмотреть список потомков корневого узла, выполним следующий код:

In [6]:
display(list(root))

[<Element 'dish' at 0x000002C4034C0EF0>,
 <Element 'dish' at 0x000002C4034E5990>]

In [7]:
display(list(root[1]))
# список потомков второго блюда в нашем меню

[<Element 'price' at 0x000002C4034E6ED0>,
 <Element 'weight' at 0x000002C4034E6BB0>,
 <Element 'class' at 0x000002C404B98EA0>]

In [9]:
display(root[0].attrib)
# атрибуты первого блюда

{'name': 'Кура'}

In [11]:
display(root[0][0]) #узел первого блюда
display(root[0][0].text) # значение узла price

<Element 'price' at 0x000002C4034C1350>

'40'

In [12]:
display(root.tag) # наименованеи тега корневого узла

'menu'

In [13]:
display(root[0][2])

<Element 'class' at 0x000002C4034E6E30>

На этом шаге мы решим задачу вывода на экран наименование всех блюд из меню, а также информацию о них (иными словами, нам необходимо обойти дерево и вывести на экран значения его листьев).

Используя цикл for, автоматизируем обход дерева. Для этого напишем следующий код:

In [16]:
for dish in root:
    for param in dish:
        print(dish.attrib['name'], param.tag, param.text)
    print()

Кура price 40
Кура weight 300
Кура class Мясо

Греча price 20
Греча weight 200
Греча class Крупа



1. В первом (внешнем) цикле перебираем потомков корня дерева (root). Потомки перебираются последовательно при помощи переменной dish. Это отдельные блюда из меню.
2. Во втором (вложенном) цикле аналогичным образом перебираем потомков каждого блюда. Этими потомками являются параметры блюда — его цена (price), вес (weight) и класс (class).
3. После этого выводим на экран название блюда (значение атрибута name), название очередного параметра (tag) и его значение (text).
4. Дополнительная функция print() в цикле верхнего уровня предназначена для организации более удобного восприятия информации — между отдельными блюдами будет выведена пустая строка.

#### ЗАГРУЖАЕМ ДАННЫЕ ИЗ XML-ФАЙЛА В DATAFRAME

Реализуем следующий алгоритм:

1. Загрузить XML-структуру файла menu.xml в переменную root.
2. Создать пустой список df_list (в него будем добавлять строчки итоговой таблицы).
3. Заранее создать список column_names с именами столбцов — название блюда (name), его цена (price), вес (weight) и класс (class).
4. В цикле организовать обход xml-дерева из корня по всем потомкам.
5. На каждой итерации цикла сформировать в виде списка строку таблицы, содержащую информацию: наименование блюда (атрибут name узла dish) и значения потомков этого узла — узлов price, weight, class.
6. Добавить сформированную строку в список df_list, используя метод append().
7. Сформировать из вложенного списка DataFrame. Имена для столбцов взять из списка column_names.

In [18]:
import pandas as pd
column_names = ['name', 'price', 'weight', 'class']
df_list = []

for dish in root:
    row = [dish.attrib['name'], dish[0].text, dish[1].text, dish[2].text]
    df_list.append(row)
    df = pd.DataFrame(df_list, columns=column_names)
display(df)

,name,price,weight,class
0,Кура,40,300,Мясо
1,Греча,20,200,Крупа


СОЗДАЁМ XML-ФАЙЛ

In [19]:
new_root = ET.Element('menu')
display(new_root)

<Element 'menu' at 0x000002C4034C3470>

Добавим в наше меню двух потомков корневого узла, которые будут представлять два блюда, то есть будут узлами dish:

In [20]:
dish1 = ET.SubElement(new_root, 'dish', name='Кура')
dish2 = ET.SubElement(new_root, 'dish', name='Греча')
display(list(new_root))

[<Element 'dish' at 0x000002C404B17330>,
 <Element 'dish' at 0x000002C403396430>]

In [21]:
price1 = ET.SubElement(dish1, "price").text = "40"
weight1 = ET.SubElement(dish1, "weight").text = "300"
class1 = ET.SubElement(dish1, "class").text = "Мясо"
display(list(dish1))

price2 = ET.SubElement(dish2, "price").text = "20"
weight2 = ET.SubElement(dish2, "weight").text = "200"
class2 = ET.SubElement(dish2, "class").text = "Крупа"
display(list(dish2))

[<Element 'price' at 0x000002C403396BB0>,
 <Element 'weight' at 0x000002C40343EC50>,
 <Element 'class' at 0x000002C40361FF10>]

[<Element 'price' at 0x000002C40361FA60>,
 <Element 'weight' at 0x000002C4048E0860>,
 <Element 'class' at 0x000002C4048E04A0>]

In [22]:
for dish in new_root:
    for param in dish:
        print(dish.attrib['name'], param.tag, param.text)
    print()

Кура price 40
Кура weight 300
Кура class Мясо

Греча price 20
Греча weight 200
Греча class Крупа



СОХРАНЕНИЕ XML-ФАЙЛА

In [24]:
new_root_string = ET.tostring(new_root)
with open("data/new_menu.xml", "wb") as f:
    f.write(new_root_string)

In [26]:
ET.ElementTree(new_root).write('new_menu_good.xml', encoding="utf-8")